In [1]:
import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import layers
from keras.preprocessing.image import DirectoryIterator, load_img, img_to_array

# Directory paths
train_dir = '/Users/tianze/cs4243_lab/miniproject/drive-download-20230925T130828Z-001/train/'
test_dir = '/Users/tianze/cs4243_lab/miniproject/drive-download-20230925T130828Z-001/test/'

# Gabor filter preprocessing
def custom_preprocessing_Gabor(img):
    # Convert the image from float [0,1] to uint8 [0,255]
    img = (img * 255).astype(np.uint8)
    # Convert to grey scale b4 Gabor
    if img.shape[-1] == 3:
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    '''
    # Apply 4 Gabor filter 0, 45, 90, 135
    g_kernel0 = cv2.getGaborKernel((21, 21), 8.0, 0, 10.0, 0.5, 0, ktype=cv2.CV_32F)
    filtered_img0 = cv2.filter2D(img, cv2.CV_8UC3, g_kernel0)
    g_kernel45 = cv2.getGaborKernel((21, 21), 8.0, np.pi/4, 10.0, 0.5, 0, ktype=cv2.CV_32F)
    filtered_img45 = cv2.filter2D(img, cv2.CV_8UC3, g_kernel45)
    g_kernel90 = cv2.getGaborKernel((21, 21), 8.0, np.pi/2, 10.0, 0.5, 0, ktype=cv2.CV_32F)
    filtered_img90 = cv2.filter2D(img, cv2.CV_8UC3, g_kernel90)
    g_kernel135 = cv2.getGaborKernel((21, 21), 8.0, np.pi/4*3, 10.0, 0.5, 0, ktype=cv2.CV_32F)
    filtered_img135 = cv2.filter2D(img, cv2.CV_8UC3, g_kernel135)
    # Stack all 4 filtered img into an img with 4 channel
    stacked_img = np.stack((filtered_img0, filtered_img45, filtered_img90, filtered_img135), axis=-1)
    '''
    # Apply 3 Gabor filter 0, 45, 90, 135
    g_kernel0 = cv2.getGaborKernel((21, 21), 8.0, 0, 10.0, 0.5, 0, ktype=cv2.CV_32F)
    filtered_img0 = cv2.filter2D(img, cv2.CV_8UC3, g_kernel0)
    g_kernel45 = cv2.getGaborKernel((21, 21), 8.0, np.pi/4, 10.0, 0.5, 0, ktype=cv2.CV_32F)
    filtered_img45 = cv2.filter2D(img, cv2.CV_8UC3, g_kernel45)
    g_kernel90 = cv2.getGaborKernel((21, 21), 8.0, np.pi/4*3, 10.0, 0.5, 0, ktype=cv2.CV_32F)
    filtered_img90 = cv2.filter2D(img, cv2.CV_8UC3, g_kernel90)
    # Stack all 3 filtered img into an img with 4 channel
    stacked_img = np.stack((filtered_img0, filtered_img45, filtered_img90), axis=-1)
    # Convert the image back to float [0,1]
    final_img = stacked_img.astype(np.float32) / 255.0
    # Ensure values are between 0 and 1
    final_img = np.clip(final_img, 0, 1)
    return final_img

train_datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocessing_Gabor,
    rescale=1./255,
    rotation_range=10,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(
    preprocessing_function=custom_preprocessing_Gabor,
    rescale=1./255,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=True
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary',
    shuffle=False
)

# Building the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Compiling the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Training the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=10,
    validation_data=test_generator,
    validation_steps=len(test_generator)
)

early_stopping = EarlyStopping(monitor='val_loss', patience=2)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Optionally, you could evaluate the model
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')

Found 3535 images belonging to 2 classes.
Found 1699 images belonging to 2 classes.
Epoch 1/10
177/177 [==============================] - 188s 1s/step - loss: 0.6826 - accuracy: 0.5825 - val_loss: 0.6629 - val_accuracy: 0.6886
Epoch 2/10
177/177 [==============================] - 188s 1s/step - loss: 0.6807 - accuracy: 0.5825 - val_loss: 0.6470 - val_accuracy: 0.6886
Epoch 3/10
177/177 [==============================] - 183s 1s/step - loss: 0.6798 - accuracy: 0.5825 - val_loss: 0.6404 - val_accuracy: 0.6886
Epoch 4/10
177/177 [==============================] - 186s 1s/step - loss: 0.6802 - accuracy: 0.5825 - val_loss: 0.6454 - val_accuracy: 0.6886
Epoch 5/10
177/177 [==============================] - 187s 1s/step - loss: 0.6797 - accuracy: 0.5825 - val_loss: 0.6461 - val_accuracy: 0.6886
Epoch 6/10
177/177 [==============================] - 187s 1s/step - loss: 0.6798 - accuracy: 0.5825 - val_loss: 0.6508 - val_accuracy: 0.6886
Epoch 7/10
177/177 [==============================] - 226s